In [1]:
from data_storage import create_connection
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


In [2]:
connection = create_connection("../database/crypto_billionairs.db")

In [3]:
def random_forest(X_train, y_train):
    
    clf = RandomForestClassifier(criterion="entropy", min_samples_split= 0.01, min_samples_leaf= 0.005, max_depth=10, class_weight="balanced_subsample")
    print("training random forest!")
    clf.fit(X_train, y_train)
   
    return "random_forest", clf

In [4]:
def knn(X_train, y_train):
    
    neigh = KNeighborsClassifier(weights="uniform", n_neighbors=5, algorithm="ball_tree")
    print("training knn!")
    neigh.fit(X_train, y_train)

    
    return "knn", neigh

In [5]:
def support_vector_machine(X_train, y_train):
    
    svc = SVC(kernel="poly", degree=4, C=1)
    print("training svc!")
    svc.fit(X_train, y_train)
    
    return "support_vector_classifier", svc

In [6]:
def mlp(X_train, y_train):
    
    mlp = MLPClassifier(hidden_layer_sizes=(10, 10), activation="tanh", solver="lbfgs", learning_rate="constant", learning_rate_init=2e-5, tol=1e-5)
    print("training mlp!")
    mlp.fit(X_train, y_train)
    
    return "mlp_classifier", mlp

In [7]:
def logistic_regression(X_train, y_train):
    
    lg = LogisticRegression(solver="liblinear", penalty="l1", C=1)
    print("training lr!")
    lg.fit(X_train, y_train)

    
    return "logistic_regression", lg
    

In [8]:
def model_ensemble(rf_model, knn_model, mlp_model, lr_model, X_train, y_train):
    
    ensemble_dataset = pd.DataFrame(columns = range(12))
    ensemble_dataset.columns = ["rf_1", "rf_0", "rf_negative_1", "knn_1", "knn_0", "knn_negative_1", "mlp_1", "mlp_0", "mlp_negative_1", "lr_1", "lr_0", "lr_negative_1"]
        
    rf_pred_proba = rf_model.predict_proba(X_train)
    knn_pred_proba = knn_model.predict_proba(X_train)
    mlp_pred_proba = mlp_model.predict_proba(X_train)
    lr_pred_proba = lr_model.predict_proba(X_train)
        
    ensemble_dataset["rf_1"] = rf_pred_proba[:, 0]
    ensemble_dataset["rf_0"] = rf_pred_proba[:, 1]
    ensemble_dataset["rf_negative_1"] = rf_pred_proba[:, 2]
        
    ensemble_dataset["knn_1"] = knn_pred_proba[:, 0]
    ensemble_dataset["knn_0"] = knn_pred_proba[:, 1]
    ensemble_dataset["knn_negative_1"] = knn_pred_proba[:, 2]
        
    ensemble_dataset["mlp_1"] = mlp_pred_proba[:, 0]
    ensemble_dataset["mlp_0"] = mlp_pred_proba[:, 1]
    ensemble_dataset["mlp_negative_1"] = mlp_pred_proba[:, 2]
        
    ensemble_dataset["lr_1"] = lr_pred_proba[:, 0]
    ensemble_dataset["lr_0"] = lr_pred_proba[:, 1]
    ensemble_dataset["lr_negative_1"] = lr_pred_proba[:, 2]

    
    print("training ensemble!")
    
    # mlp = MLPClassifier(hidden_layer_sizes=(10, 10), activation="tanh", solver="lbfgs", learning_rate="constant", learning_rate_init=2e-5, tol=1e-5)
    # mlp.fit(ensemble_dataset, y_train)
    
    clf_new = RandomForestClassifier(criterion="entropy")
    clf_new.fit(X_train, y_train)
    
    return "ensemble", clf_new
    
    

In [9]:
def evaluation(X_test, y_test, model, model_name, table, db_connection):
    
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)
    
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    
    df.to_sql(f"{table}_{model_name}_pooling", db_connection, if_exists="replace")
    
    return f"{model_name}_pooling", f1score, f1score_macro, recall, precision, y_pred_proba[:, 0], y_pred_proba[:, 1], y_pred_proba[:, 2]

In [13]:
def evaluation_ensemble(market_data, X_test, y_test, model, model_name, table, db_connection):
    
    y_pred = model.predict(X_test)
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([market_data])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    
    df.to_sql(f"{table}_{model_name}_pooling", db_connection, if_exists="replace")
    
    return f"{model_name}_pooling", f1score, f1score_macro, recall, precision

In [14]:
def apply_ml_algorithms_pooling(db_connection):
    
    df_temp = pd.read_sql_query(f"select * from cryptocurrency_pooling_dataset", db_connection)
    df_temp = shuffle(df_temp, random_state=42069)
    
    y_train = df_temp["buy_indicator"] + df_temp["short_indicator"]
    y_train = y_train.fillna(0)
    y_train = y_train.astype(str)
        
    X_train = df_temp.drop(["return", "buy_indicator", "short_indicator","close_buy_indicator", "close_short_indicator", "time", "index", "level_0"], axis=1)
    ros = RandomOverSampler(random_state=0)
    X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
    
    rf_name, rf_model = random_forest(X_resampled, y_resampled)
    knn_name, knn_model = knn(X_resampled, y_resampled)
    #svc_name, svc_model = support_vector_machine(X_resampled, y_resampled)
    mlp_name, mlp_model = mlp(X_resampled, y_resampled)
    lr_name, lr_model = logistic_regression(X_resampled, y_resampled)
    
    ensemble_name, ensemble_model = model_ensemble(rf_model, knn_model, mlp_model, lr_model, X_train, y_train)
    #creating the evaluation metric
    df_ml = pd.DataFrame(columns = range(6))
    df_ml.columns = ["table_name", "model", "f1-score weighted", "f1-score macro", "recall macro", "precision macro"]
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "_1day_features" in name and 'trades' not in name and 'equity_curve' not in name and '_pooling' not in name]
    print(filtered_table_names)
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"select * from {table}", db_connection)
        y = df_temp["buy_indicator"] + df_temp["short_indicator"]
        y = y.fillna(0)
        y = y.astype(str)
        X = df_temp.drop(["return", "buy_indicator", "short_indicator","close_buy_indicator", "close_short_indicator", "time", "index"], axis=1)

        X_test = X.iloc[-365:]
        y_test = y.iloc[-365:]
        
        string, score, f1score_macro, recall, precision, rf_pred_class1, rf_pred_class0, rf_pred_class_negative_1 = evaluation(X_test, y_test, rf_model, rf_name, table, connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision, knn_pred_class1, knn_pred_class0, knn_pred_class_negative_1 = evaluation(X_test, y_test, knn_model, knn_name, table, connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        # string, score, f1score_macro, recall, precision = evaluation(X_test, y_test, svc_model, svc_name, table, connection)
        # df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision, mlp_pred_class1, mlp_pred_class0, mlp_pred_class_negative_1 = evaluation(X_test, y_test, mlp_model, mlp_name, table, connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision, lr_pred_class1, lr_pred_class0, lr_pred_class_negative_1 = evaluation(X_test, y_test, lr_model, lr_name, table, connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        ensemble_dataset = pd.DataFrame(columns = range(12))
        ensemble_dataset.columns = ["rf_1", "rf_0", "rf_negative_1", "knn_1", "knn_0", "knn_negative_1", "mlp_1", "mlp_0", "mlp_negative_1", "lr_1", "lr_0", "lr_negative_1"]
        ensemble_dataset["rf_1"] = rf_pred_class1
        ensemble_dataset["rf_0"] = rf_pred_class0
        ensemble_dataset["rf_negative_1"] = rf_pred_class_negative_1
        
        ensemble_dataset["knn_1"] = knn_pred_class1
        ensemble_dataset["knn_0"] = knn_pred_class0
        ensemble_dataset["knn_negative_1"] = knn_pred_class_negative_1
        
        ensemble_dataset["mlp_1"] = mlp_pred_class1
        ensemble_dataset["mlp_0"] = mlp_pred_class0
        ensemble_dataset["mlp_negative_1"] = mlp_pred_class_negative_1
        
        ensemble_dataset["lr_1"] = lr_pred_class1
        ensemble_dataset["lr_0"] = lr_pred_class0
        ensemble_dataset["lr_negative_1"] = lr_pred_class_negative_1
        
        string, score, f1score_macro, recall, precision = evaluation_ensemble(X_test, ensemble_dataset, y_test, ensemble_model, ensemble_name, table, connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
    return df_ml
    

In [15]:
df_ml = apply_ml_algorithms_pooling(connection)

training random forest!
training knn!
training mlp!


c:\Users\janfa\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


training lr!
training ensemble!
['ADA_1min_complete_1day_preprocessed_1day_features', 'BCH_1min_complete_1day_preprocessed_1day_features', 'BTC_1min_complete_1day_preprocessed_1day_features', 'DOGE_1min_complete_1day_preprocessed_1day_features', 'ETH_1min_complete_1day_preprocessed_1day_features', 'LINK_1min_complete_1day_preprocessed_1day_features', 'LTC_1min_complete_1day_preprocessed_1day_features', 'TRX_1min_complete_1day_preprocessed_1day_features']
[[0.13548467 0.55631022 0.30820511]
 [0.15838687 0.5428805  0.29873263]
 [0.18772939 0.5699501  0.24232051]
 ...
 [0.3043268  0.25538276 0.44029044]
 [0.3144043  0.26033842 0.42525728]
 [0.33884449 0.30855288 0.35260264]]
[[0.  0.2 0.8]
 [0.  0.2 0.8]
 [0.  0.2 0.8]
 ...
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]]
[[0.32129683 0.30896826 0.36973491]
 [0.32129683 0.30896826 0.36973491]
 [0.32129683 0.30896826 0.36973491]
 ...
 [0.38916849 0.27739336 0.33343815]
 [0.38916849 0.27739336 0.33343815]
 [0.38916849 0.27739336 0.33343815]]
[

c:\Users\janfa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\janfa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\janfa\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- knn_0
- knn_1
- knn_negative_1
- lr_0
- lr_1
- ...
Feature names seen at fit time, yet now missing:
- adl
- adx14
- ao14
- atrp14
- close
- ...

  warnings.warn(message

ValueError: X has 12 features, but RandomForestClassifier is expecting 25 features as input.

In [ ]:
df_ml

,table_name,model,f1-score weighted,f1-score macro,recall macro,precision macro
0,ADA_1min_complete_1day_preprocessed_1day_features,random_forest_pooling,0.602278,0.420879,0.456236,0.423721
1,ADA_1min_complete_1day_preprocessed_1day_features,knn_pooling,0.051253,0.127965,0.312343,0.083177
2,ADA_1min_complete_1day_preprocessed_1day_features,mlp_classifier_pooling,0.045408,0.104690,0.315163,0.070403
3,ADA_1min_complete_1day_preprocessed_1day_features,logistic_regression_pooling,0.365707,0.291234,0.421558,0.419785
4,ADA_1min_complete_1day_preprocessed_1day_features,ensemble_pooling,0.632125,0.380488,0.376806,0.409230
5,BCH_1min_complete_1day_preprocessed_1day_features,random_forest_pooling,0.562450,0.388222,0.455083,0.396363
6,BCH_1min_complete_1day_preprocessed_1day_features,knn_pooling,0.147135,0.174320,0.365642,0.449029
7,BCH_1min_complete_1day_preprocessed_1day_features,mlp_classifier_pooling,0.689792,0.384725,0.377463,0.454608
8,BCH_1min_complete_1day_preprocessed_1day_features,logistic_regression_pooling,0.575483,0.372043,0.404649,0.376522
9,BCH_1min_complete_1day_preprocessed_1day_features,ensemble_pooling,0.640825,0.383473,0.383373,0.448388
